In [32]:
import pandas as pd
data = pd.read_csv("../EBOOK-RATING-WITH-USERS-AND-TITLE.csv", delimiter=";")
print(data.head())

     id                                              judul  \
0  1197  EDEN IN THE EAST SURGA DI TIMUR,STEPHEN OPPENH...   
1  1209  24416 NASKAH DEPARTEMEN TENTANG MATA KULIAH MA...   
2  1210  24418 NASKAH DEPARTEMEN TENTANG MATA KULIAH MA...   
3  1234  BUKU PETUNJUK PELAKSANAAN PEDOMAN PENGAMANAN P...   
4  1236  BUKU PETUNJUK PELAKSANAAN PEDOMAN PENYIAPAN SA...   

                                           deskripsi  user_id  rating  
0  Buku ini diterbitkan untuk menambah wawasan pa...      311     5.0  
1  Buku ini diterbitkan untuk menambah wawasan pa...     1631     4.0  
2  Buku ini diterbitkan untuk menambah wawasan pa...     1631     5.0  
3  Buku ini diterbitkan untuk menambah wawasan pa...     1994     2.0  
4  Buku ini diterbitkan untuk menambah wawasan pa...     3090     5.0  


In [35]:
from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [41]:
content_df = data[['id', 'judul', 'deskripsi']]
content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

TypeError: 'DatasetAutoFolds' object is not subscriptable

In [38]:
# Use TF-IDF vectorizer to convert content into a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
content_matrix = tfidf_vectorizer.fit_transform(content_df['Content'])

content_similarity = linear_kernel(content_matrix, content_matrix)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data[['user_id', 
                                  'id', 
                                  'rating']], reader)

def get_content_based_recommendations(product_id, top_n):
    index = content_df[content_df['id'] == product_id].index[0]
    similarity_scores = content_similarity[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommendations = content_df.loc[similar_indices, 'id'].values
    return recommendations

TypeError: 'DatasetAutoFolds' object is not subscriptable

In [45]:
get_content_based_recommendations(1236, 3)

array([1268])

In [47]:
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions = algo.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    return recommendations

In [48]:
def get_hybrid_recommendations(user_id, product_id, top_n):
    content_based_recommendations = get_content_based_recommendations(product_id, top_n)
    collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
    hybrid_recommendations = list(set(content_based_recommendations + collaborative_filtering_recommendations))
    return hybrid_recommendations[:top_n]

In [50]:
get_hybrid_recommendations(311, 1197, 10)

[2691, 2727, 2700, 5646, 2738, 2675, 2711, 2745, 2685, 2751]